In [5]:
# @title MSR_output
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# Global exclusion list and available months
exclude_months = []
available_months = []

# ---------- Interactive Widgets ----------
month_selector = widgets.SelectMultiple(
    options=[],  # Populated later
    description='Exclude Months:',
    layout=widgets.Layout(width='50%'),
    style={'description_width': 'initial'}
)

apply_button = widgets.Button(description="Apply Exclusion")
dropdown = widgets.Dropdown(
    options=[
        'CMSP Trending',
        'Remedy Ticket Trending',
        'CRQ Trending',
        'CEWA',
        'Call Queue Hygiene',
        'Monthly Audits'
    ],
    description='Graph:',
)
plot_button = widgets.Button(description="Generate Graph")
output = widgets.Output()


# ---------- Core Plotting Functions ----------

def generate_combo(a, b):
    mask = ~a.index.isin(exclude_months)
    fig, ax1 = plt.subplots(figsize=(10, 5))
    bar_width = 0.35
    bars = ax1.bar(a.index[mask], a['Submitted Overall'][mask], bar_width, alpha=0.3, color='aqua')
    ax1.set_ylabel('Submitted overall')
    ax2 = ax1.twinx()
    ax2.plot(a.index[mask], a['Resolved Percentage'][mask], marker='s', color='blue')
    ax2.set_ylabel('Resolved %')
    ax2.set_yticks(np.arange(0, 120, 10))

    for x, y in zip(a.index[mask], a['Submitted Overall'][mask]):
        ax1.text(x, y + 3, str(y), ha='center', fontsize=11, fontweight='bold')
    for x, y in zip(a.index[mask], a['Resolved Percentage'][mask]):
        ax2.text(x, y - 5, str(y), ha='center', fontsize=10)

    plt.title(b)
    plt.xticks(rotation=45, ha='right')
    fig.legend([ax2.get_lines()[0], bars], ['Resolved %', 'Submitted Overall'],
               loc='center left', bbox_to_anchor=(0.6, 1.0), fontsize=10)
    plt.tight_layout()
    plt.subplots_adjust(right=0.8)
    plt.show()


def plot_crq():
    df = pd.read_excel('https://raw.githubusercontent.com/roypromit/MSR/main/MSR_Data.xlsx', sheet_name='Changes', engine='openpyxl')
    df['Month_dt'] = pd.to_datetime(df['Month'], format="%b'%y")
    grouped = df.groupby('Month_dt')[['Changes']].sum()
    grouped.index = grouped.index.strftime("%b'%y")
    mask = ~grouped.index.isin(exclude_months)
    bar_width = 0.35
    plt.figure(figsize=(10, 5))
    plt.bar(grouped.index[mask], grouped['Changes'][mask], bar_width, color='aqua')
    for x, y in zip(grouped.index[mask], grouped['Changes'][mask]):
        plt.text(x, y + 0.1, str(y), ha='center', fontsize=11, fontweight='bold')
    plt.title('CRQ Trending')
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.yticks([])
    plt.show()


def plot_cewa():
    df = pd.read_excel('https://raw.githubusercontent.com/roypromit/MSR/main/MSR_Data.xlsx', sheet_name='CEWA', engine='openpyxl')
    df['Month'] = pd.to_datetime(df['Month'], format="%b'%y").dt.strftime("%b'%y")
    mask = ~df['Month'].isin(exclude_months)
    bar_width = 0.35
    plt.figure(figsize=(10, 5))
    bars = plt.bar(df['Month'][mask], df['CEWA'][mask], bar_width, color='aqua')
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, height + 0.2, str(height), ha='center', va='bottom', fontsize=11, fontweight='bold')
    plt.title('CEWA Trending')
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.yticks([])
    plt.show()


def plot_call_queue():
    df = pd.read_excel('https://raw.githubusercontent.com/roypromit/MSR/main/MSR_Data.xlsx', sheet_name='Call queue hygiene', engine='openpyxl')
    df['Month'] = pd.to_datetime(df['Month'], format="%b'%y").dt.strftime("%b'%y")
    mask = ~df['Month'].isin(exclude_months)
    bar_width = 0.2
    index = np.arange(len(df['Month'][mask]))
    fig, ax = plt.subplots(figsize=(10, 5))
    bar1 = ax.bar(index - bar_width, df['No. of Calls'][mask], bar_width, label='Calls', color='aqua')
    bar2 = ax.bar(index, df['RONA'][mask], bar_width, label='No Answer', color='orange')
    bar3 = ax.bar(index + bar_width, df['Aban Ring'][mask], bar_width, label='Abandoned', color='lightgreen')
    for bar in bar1 + bar2 + bar3:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, height + 0.2, str(height), ha='center', va='bottom', fontsize=11, fontweight='bold')
    ax.set_title('Call Queue Hygiene Metrics')
    ax.set_xticks(index)
    ax.set_xticklabels(df['Month'][mask], rotation=0)
    ax.legend()
    plt.tight_layout()
    plt.yticks([])
    plt.show()


def plot_audits():
    df = pd.read_excel('https://raw.githubusercontent.com/roypromit/MSR/main/MSR_Data.xlsx', sheet_name='Monthly Audits', engine='openpyxl')
    df['Month'] = pd.to_datetime(df['Month'], format="%b'%y").dt.strftime("%b'%y")
    mask = ~df['Month'].isin(exclude_months)
    bar_width = 0.35
    plt.figure(figsize=(10, 5))
    bars = plt.bar(df['Month'][mask], df['Number of audits'][mask], bar_width, color='aqua')
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, height + 0.2, str(height), ha='center', va='bottom', fontsize=11, fontweight='bold')
    plt.title('Number of Audits')
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.yticks([])
    plt.show()


def plot_graph(option):
    if option == "CMSP Trending":
        data = pd.read_excel('https://raw.githubusercontent.com/roypromit/MSR/main/MSR_Data.xlsx', sheet_name='CMSP Trending', engine='openpyxl')
        data['Month_dt'] = pd.to_datetime(data['Month'], format="%b'%y")
        grouped = data.groupby('Month_dt')[['Submitted Overall', 'Resolved Percentage']].sum()
        grouped.index = grouped.index.strftime("%b'%y")
        generate_combo(grouped, "CMSP Trending")
    elif option == "Remedy Ticket Trending":
        data = pd.read_excel('https://raw.githubusercontent.com/roypromit/MSR/main/MSR_Data.xlsx', sheet_name='Remedy Ticket Trending', engine='openpyxl')
        data['Month_dt'] = pd.to_datetime(data['Month'], format="%b'%y")
        grouped = data.groupby('Month_dt')[['Submitted Overall', 'Resolved Percentage']].sum()
        grouped.index = grouped.index.strftime("%b'%y")
        generate_combo(grouped, "Remedy Ticket Trending")
    elif option == "CRQ Trending":
        plot_crq()
    elif option == "CEWA":
        plot_cewa()
    elif option == "Call Queue Hygiene":
        plot_call_queue()
    elif option == "Monthly Audits":
        plot_audits()
    else:
        print("Invalid option")


# ---------- Interactive Control Functions ----------

def load_months():
    global available_months
    urls = [
        ('CMSP Trending', 'Month'),
        ('Remedy Ticket Trending', 'Month'),
        ('Changes', 'Month'),
        ('CEWA', 'Month'),
        ('Call queue hygiene', 'Month'),
        ('Monthly Audits', 'Month'),
    ]
    month_set = set()
    for sheet, col in urls:
        df = pd.read_excel('https://raw.githubusercontent.com/roypromit/MSR/main/MSR_Data.xlsx', sheet_name=sheet, engine='openpyxl')
        df[col] = pd.to_datetime(df[col], format="%b'%y").dt.strftime("%b'%y")
        month_set.update(df[col].unique())
    available_months = sorted(month_set, key=lambda x: pd.to_datetime(x, format="%b'%y"))
    month_selector.options = available_months


def apply_exclusions(b):
    global exclude_months
    exclude_months = list(month_selector.value)
    with output:
        clear_output()
        print(f"Excluded months updated: {exclude_months}")


def on_plot_click(b):
    with output:
        clear_output(wait=True)
        plot_graph(dropdown.value)


# ---------- Setup and Display ----------
apply_button.on_click(apply_exclusions)
plot_button.on_click(on_plot_click)

load_months()

display(widgets.VBox([
    widgets.Label("Select months to exclude and generate graphs:"),
    month_selector,
    apply_button,
    dropdown,
    plot_button,
    output
]))


HTTPError: HTTP Error 404: Not Found